## Detecting Faces and Saving the Faces

In [1]:
import cv2
import os
import numpy as np

# Load the Haar Cascade
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Function to detect faces and draw bounding boxes
def detect_bounding_box(vid):
    gray_image = cv2.cvtColor(vid, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_image, 1.1, 5, minSize=(40, 40))
    for (x, y, w, h) in faces:
        cv2.rectangle(vid, (x, y), (x + w, y + h), (0, 255, 0), 2)
    return faces

# Function to save detected faces in a folder
def save_detected_faces(vid, faces, folder_path, saved_faces):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    face_id = len(os.listdir(folder_path))  # Start from the current number of files in the folder
    for (x, y, w, h) in faces:
        new_face = True
        for (saved_x, saved_y, saved_w, saved_h) in saved_faces:
            distance = np.linalg.norm([x - saved_x, y - saved_y])
            if distance < 50 and abs(w - saved_w) < 20 and abs(h - saved_h) < 20:
                new_face = False
                break
        if new_face:
            face_image = vid[y:y+h, x:x+w]
            face_path = os.path.join(folder_path, f"face_{face_id}.jpg")
            cv2.imwrite(face_path, face_image)
            saved_faces.append((x, y, w, h))
            face_id += 1

# Initialize video capture
video_capture = cv2.VideoCapture(0)

# Create a folder to store detected faces
folder_path = "faces_detected"
saved_faces = []

while True:
    result, video_frame = video_capture.read()
    if not result:
        break

    faces = detect_bounding_box(video_frame)  # Detect faces
    save_detected_faces(video_frame, faces, folder_path, saved_faces)  # Save detected faces

    cv2.imshow("Face Detection", video_frame)  # Display the processed frame

    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
    if cv2.getWindowProperty("Face Detection", cv2.WND_PROP_VISIBLE) < 1:
        break

video_capture.release()
cv2.destroyAllWindows()


In [5]:
cv2.destroyAllWindows()     # Closes all OpenCV windows
video_capture.release()     # Releases the video capture object, closing the connection to the camera
